In [1]:
import cv2 as cv       #v3.4.2
import pandas as pd    #v1.0.3
import os 
from openpyxl import load_workbook

### Function manualLocate 
manualLocate marks the image at the location of a left mouse click and saves the x,y coordinates of the clicked location

In [3]:
def manualLocate(event, x, y, flags, param):
    global points
    
    if event == cv.EVENT_LBUTTONDOWN:
        points.append([x,y])
        cv.drawMarker(editImg, (x,y), (255,0,0), cv.MARKER_CROSS,10,2)

### Function rectangle
rectangle stores the coordinates of a rectangle and displays the box drawn by the user by clicking and dragging 

In [ ]:
def rectangle(event, x, y, flags, param):
    global boxesS
    global boxesE
    
    if event ==cv.EVENT_LBUTTONDOWN:
        sbox = (x, y)
        boxesS.append(sbox)
    
    elif event == cv.EVENT_LBUTTONUP:
        ebox= (x, y)
        boxesE.append(ebox)
        cv.rectangle(rectImg,pt1=boxesS[-1],pt2=boxesE[-1],color=[255,0,0],thickness=2)

## Main Code

The following code prompts the user to perform manual edits to the automated image analysis results. First, the user is prompted to click on the microtubules that were not identified by the automated procedure. Next, the user is prompted to draw rectangles around false positives to remove them. 

In [ ]:
outputDirectory = 'D:\PLMLAutoResults' #directory containing the labeled images from the automated analysis
inputDirectory = 'D:\ManualInputFolder' #directory containing all of the images that need to be edited manually
excelInFile = "AutoAlignedResultsPLML.xls" #Excel file containing automated analysis results
finalDirectory = 'D:\PLMLManualResults' #directory containing all of the new manually edited, labeled images
excelOutFile = "ManualAlignedResultsPLML.xlsx" #Excel file where the results of the manual edits will be stored

resize_factor = 1.5
dfResults = []

for imageFile in os.listdir(inputDirectory):
    #read in automatically identified points
    df = pd.read_excel(excelInFile, sheet_name=imageFile[19:])
    points = df.values.tolist()

    #show image with auto-labeled points and original unlabeled image
    original = cv.imread(os.path.join(inputDirectory,imageFile))
    if os.path.exists(os.path.join(outputDirectory,imageFile)):
        editImg = cv.imread(os.path.join(outputDirectory,imageFile))
    else:
        editImg = original

    #original unlabeled image
    cv.namedWindow('Original', cv.WINDOW_NORMAL)
    h,w = original.shape[:2]
    h = int(h / resize_factor)
    w = int(w / resize_factor)
    cv.resizeWindow('Original', w, h)

    #auto labeled image
    cv.namedWindow("AddPts", cv.WINDOW_NORMAL)
    h2,w2 = editImg.shape[:2]
    h2 = int(h2 / resize_factor)
    w2 = int(w2 / resize_factor)
    cv.resizeWindow('AddPts', w2, h2)
    cv.setMouseCallback("AddPts",manualLocate)

    #ask user to identify missed points by clicking on the microtubules
    while True:
        cv.imshow("Original", original)

        if prevImg != None:
            cv.imshow("Previous", prevImg)

        cv.imshow("AddPts",editImg)

        key = cv.waitKey(1) & 0xFF

        if key ==ord("c"):
            break

    cv.destroyAllWindows()

    #moving on to removing false positives
    boxesS = []
    boxesE = []

    if os.path.exists(os.path.join(outputDirectory,imageFile)):
        rectImg = cv.imread(os.path.join(outputDirectory,imageFile))
    else: 
        rectImg = original

    #show original image
    cv.namedWindow('Original', cv.WINDOW_NORMAL)
    h,w = original.shape[:2]
    h = int(h / resize_factor)
    w = int(w / resize_factor)
    cv.resizeWindow('Original', w, h)

    #show auto labeled image
    cv.namedWindow("Erase", cv.WINDOW_NORMAL)
    h2,w2 = rectImg.shape[:2]
    h2 = int(h2 / resize_factor)
    w2 = int(w2 / resize_factor)
    cv.resizeWindow('Erase', w2, h2)
    cv.setMouseCallback("Erase",rectangle)

    #ask user to draw boxes around false positives to remove them
    while True:
        cv.imshow("Original", original)

        if prevImg != None:
            cv.imshow("Previous", prevImg)

        cv.imshow("Erase",rectImg)
        key = cv.waitKey(1) & 0xFF

        if key ==ord("c"):
            break

    cv.destroyAllWindows()

    for i in range(len(boxesS)):
        sx = boxesS[i][0]
        sy = boxesS[i][1]
        ex = boxesE[i][0]
        ey = boxesE[i][1]

        for point in points:
            x = point[0]
            y = point[1]
            if (x>sx and x<ex) and (y>sy and y<ey):
                points.remove(point)

    #append microtubule location data for this image to automated results
    df = pd.DataFrame(points, columns = ['x','y'])
    dfResults.append((df,imageFile))

    #export new output image with manual edits
    newOutImg = cv.imread(os.path.join(inputDirectory,imageFile))

    for point in points:
        cv.drawMarker(newOutImg, (int(point[0]),int(point[1])), (0,0,255), cv.MARKER_CROSS,10,2)

    cv.imwrite(os.path.join(finalDirectory,imageFile),newOutImg)
        
#write results to Excel file

with pd.ExcelWriter(excelOutFile, engine='openpyxl') as writer:
    if os.path.exists(excelOutFile):
        book = load_workbook(excelOutFile)
        writer.book = book
        writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
    for (df,fileName) in dfResults:
        df.to_excel(writer, sheet_name=fileName[19:], index=False)

        
writer.save()